# Book data exploration

I want to just run through the book archive and see if I can find some interesting word clouds.

I think I should start by figuring out how to grab books in english first.

It looks like that is in the metadata for each of the books.

Documentation here:

https://docs.google.com/document/d/10cNGGYrDFu0BJg-pUYYzKpjB1TWkqKspTZl2YG-yLJ4/edit

In [21]:
# Import libraries here:
import requests
import xml.etree.ElementTree as ET
import re

import pandas as pd
import numpy as np
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

## Read in the identifiers here

In [4]:
df = pd.read_csv("2017-12-26_public-domain-books-dataset_800k-identifiers.csv",header=None)

In [50]:
base_url = 'https://archive.org/download/{itemid}/{itemid}_meta.txt'
first_url = base_url.format(itemid=df.loc[6900,0])

In [51]:
first_url

'https://archive.org/download/CAT31283978/CAT31283978_meta.txt'

In [52]:
r = requests.get(first_url)

TypeError: 'str' object is not callable

In [37]:
group = re.search('Yellow', r.text)

In [46]:
len(group.group())

6